In [105]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [106]:
cols = ['Sex','length','Diameter','Height','Whole_weight','Shucked_weight',
        'Viscera_weight','Shell_weight','Rings']
df = pd.read_csv('abalone/abalone.data',names=cols)
df = pd.get_dummies(df,columns=['Sex'],dtype='int')
df

,length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
4172,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11,1,0,0
4173,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10,0,0,1
4174,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9,0,0,1
4175,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10,1,0,0


In [107]:
df.isna().sum()

length            0
Diameter          0
Height            0
Whole_weight      0
Shucked_weight    0
Viscera_weight    0
Shell_weight      0
Rings             0
Sex_F             0
Sex_I             0
Sex_M             0
dtype: int64

In [108]:

newdf = df.copy()
newdf = newdf.sample(frac=1)
newdf_x = newdf[['Sex_F','Sex_I','Sex_M','length','Diameter','Height','Whole_weight','Shucked_weight',
        'Viscera_weight','Shell_weight']]
newdf_y = newdf[['Rings']]

train_x, test_x, train_y, test_y =  train_test_split(newdf_x, newdf_y, test_size=0.30)


In [109]:
train_x

,Sex_F,Sex_I,Sex_M,length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
1365,0,0,1,0.610,0.475,0.170,1.0265,0.4350,0.2335,0.3035
3589,0,0,1,0.650,0.515,0.180,1.4630,0.6580,0.3135,0.4115
3578,1,0,0,0.600,0.475,0.170,1.0880,0.4905,0.2475,0.3100
1539,0,1,0,0.355,0.260,0.090,0.1985,0.0715,0.0495,0.0580
733,0,0,1,0.505,0.380,0.130,0.6560,0.2270,0.1785,0.2200
...,...,...,...,...,...,...,...,...,...,...
205,1,0,0,0.440,0.340,0.140,0.4820,0.1860,0.1085,0.1600
2983,1,0,0,0.525,0.410,0.135,0.7905,0.4065,0.1980,0.1770
297,0,1,0,0.280,0.210,0.085,0.1065,0.0390,0.0295,0.0300
233,0,1,0,0.275,0.215,0.075,0.1155,0.0485,0.0290,0.0350


In [110]:
train_y

,Rings
1365,10
3589,11
3578,10
1539,7
733,13
...,...
205,9
2983,8
297,4
233,7


In [111]:
test_x

,Sex_F,Sex_I,Sex_M,length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
4069,0,1,0,0.455,0.335,0.105,0.4055,0.1750,0.0920,0.1185
1643,1,0,0,0.580,0.435,0.150,0.8340,0.4280,0.1515,0.2300
2314,1,0,0,0.580,0.475,0.135,0.9250,0.3910,0.1650,0.2750
966,0,1,0,0.510,0.375,0.100,0.5785,0.2380,0.1225,0.1750
2020,0,0,1,0.515,0.400,0.140,0.6335,0.2880,0.1450,0.1680
...,...,...,...,...,...,...,...,...,...,...
3884,0,1,0,0.290,0.225,0.075,0.1520,0.0710,0.0590,0.0450
1066,0,1,0,0.320,0.225,0.085,0.1415,0.0675,0.0295,0.0405
795,0,0,1,0.515,0.405,0.145,0.6950,0.2150,0.1635,0.2340
4020,1,0,0,0.680,0.550,0.175,1.4730,0.7130,0.2820,0.4295


In [112]:
test_y

,Rings
4069,8
1643,8
2314,14
966,7
2020,9
...,...
3884,9
1066,6
795,15
4020,11


In [113]:
from sklearn.ensemble import RandomForestRegressor

In [114]:
randForestReg = RandomForestRegressor()
randForestReg

RandomForestRegressor()

In [115]:

randForestReg.fit(train_x, train_y)

d:\Principal\Workspace\Python\py3114\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [116]:
randForestReg

RandomForestRegressor()

In [117]:

y_preds = randForestReg.predict(test_x)

In [118]:
y_preds

array([ 7.4 ,  9.42, 10.79, ..., 12.13, 11.5 ,  7.22])

In [119]:
y_preds = np.floor(y_preds)
y_preds

array([ 7.,  9., 10., ..., 12., 11.,  7.])

In [133]:
y_preds

array([ 7.,  9., 10., ..., 12., 11.,  7.])

In [121]:
test_y

,Rings
4069,8
1643,8
2314,14
966,7
2020,9
...,...
3884,9
1066,6
795,15
4020,11


In [122]:
from sklearn.metrics import mean_squared_error, r2_score

print("tree score", mean_squared_error(test_y, y_preds))


tree score 4.874800637958533


In [123]:
print('R2 score',randForestReg.score(test_x,test_y))

R2 score 0.5345303399961461


In [ ]:
import matplotlib.pyplot as plt

plt.plot(r2_score(test_y, np.mean(preds[:i+1], axis=0)));